In [1]:
import pandas as pd
import numpy as np


C:\Users\Shreyas Bagave\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Shreyas Bagave\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

patients = pd.read_csv("Patient_Data.csv")
billing = pd.read_csv("Billing_Data.csv")


In [3]:

print("Patient Dataset Info:")
print(patients.info())

print("\nBilling Dataset Info:")
print(billing.info())


Patient Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 464.0+ bytes
None

Billing Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 248.0 bytes
None


In [4]:

billing_relevant = patients[['PatientID', 'Department', 'Doctor', 'BillAmount']]
print("\nRelevant Patient Columns:\n", billing_relevant.head())


patients_cleaned = patients.drop(columns=['ReceptionistID', 'CheckInTime'])
print("\nAfter Dropping Admin Columns:\n", patients_cleaned.head())



Relevant Patient Columns:
    PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN

After Dropping Admin Columns:
    PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


In [5]:


dept_bill = patients_cleaned.groupby('Department')['BillAmount'].sum().reset_index()
print("\nTotal Bill Amount per Department:\n", dept_bill)


patients_unique = patients_cleaned.drop_duplicates(subset='PatientID', keep='first')
print("\nAfter Removing Duplicate Patients:\n", patients_unique)




Total Bill Amount per Department:
     Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0

After Removing Duplicate Patients:
    PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


In [6]:

mean_bill = patients_unique['BillAmount'].mean()
patients_unique['BillAmount'].fillna(mean_bill, inplace=True)
print("\nAfter Filling Missing BillAmount:\n", patients_unique)


merged_df = pd.merge(patients_unique, billing, on="PatientID", how="left")
print("\nMerged Dataset:\n", merged_df.head())


new_patients = pd.DataFrame({
    "PatientID": [101, 102],
    "Name": ["Ravi", "Anita"],
    "Department": ["Cardiology", "Neurology"],
    "Doctor": ["Dr. Sharma", "Dr. Rao"],
    "BillAmount": [15000, 20000]
})



After Filling Missing BillAmount:
    PatientID     Name   Department     Doctor   BillAmount
0        101    Alice   Cardiology  Dr. Smith  5000.000000
1        102      Bob    Neurology   Dr. John  6233.333333
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000
3        104    David   Cardiology  Dr. Smith  6200.000000
4        105      Eva  Dermatology   Dr. Rose  6233.333333

Merged Dataset:
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  


C:\Users\Shreyas Bagave\AppData\Local\Temp\ipykernel_21088\582364730.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  patients_unique['BillAmount'].fillna(mean_bill, inplace=True)
C:\Users\Shreyas Bagave\AppData\Local\Temp\ipykernel_21088\582364730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_unique['BillAmount'].fillna(mean

In [7]:

patients_final = pd.concat([merged_df, new_patients], ignore_index=True)
print("\nAfter Adding New Patients:\n", patients_final)


new_billing_cols = pd.DataFrame({
    "InsuranceCovered": [True, False, True, False, True, False, True],
    "FinalAmount": [12000, 25000, 18000, 30000, 22000, 21000, 27000]
})



After Adding New Patients:
    PatientID     Name   Department      Doctor    BillAmount  \
0        101    Alice   Cardiology   Dr. Smith   5000.000000   
1        102      Bob    Neurology    Dr. John   6233.333333   
2        103  Charlie  Orthopedics     Dr. Lee   7500.000000   
3        104    David   Cardiology   Dr. Smith   6200.000000   
4        105      Eva  Dermatology    Dr. Rose   6233.333333   
5        101     Ravi   Cardiology  Dr. Sharma  15000.000000   
6        102    Anita    Neurology     Dr. Rao  20000.000000   

   InsuranceCovered  FinalAmount  
0            2000.0       3000.0  
1            1500.0       3500.0  
2            2500.0       5000.0  
3            3000.0       3200.0  
4            1000.0       4000.0  
5               NaN          NaN  
6               NaN          NaN  


In [9]:


patients_final = pd.concat([patients_final, new_billing_cols], axis=1)
print("\nFinal Dataset with New Billing Columns:\n", patients_final)

print("\n Final Cleaned Dataset Ready for Analysis ")



Final Dataset with New Billing Columns:
    PatientID     Name   Department      Doctor    BillAmount  \
0        101    Alice   Cardiology   Dr. Smith   5000.000000   
1        102      Bob    Neurology    Dr. John   6233.333333   
2        103  Charlie  Orthopedics     Dr. Lee   7500.000000   
3        104    David   Cardiology   Dr. Smith   6200.000000   
4        105      Eva  Dermatology    Dr. Rose   6233.333333   
5        101     Ravi   Cardiology  Dr. Sharma  15000.000000   
6        102    Anita    Neurology     Dr. Rao  20000.000000   

   InsuranceCovered  FinalAmount  InsuranceCovered  FinalAmount  \
0            2000.0       3000.0              True        12000   
1            1500.0       3500.0             False        25000   
2            2500.0       5000.0              True        18000   
3            3000.0       3200.0             False        30000   
4            1000.0       4000.0              True        22000   
5               NaN          NaN           